In [ ]:
import pandas as pd

# Load the existing data
df = pd.read_csv('project_ideas.csv')

# Identify duplicate rows based on the 'idea' column
duplicates = df[df.duplicated(subset=['idea'], keep=False)]

# Display duplicate rows
# import ace_tools as tools; tools.display_dataframe_to_user(name="Duplicate Rows", dataframe=duplicates)

# Drop duplicates, keeping the first occurrence
df_cleaned = df.drop_duplicates(subset=['idea'], keep='first').reset_index(drop=True)

# # Display cleaned DataFrame sample
# tools.display_dataframe_to_user(name="Cleaned Data (First 10 Rows)", dataframe=df_cleaned.head(10))

# Save the cleaned file
cleaned_path = 'project_ideas_cleaned.csv'
df_cleaned.to_csv(cleaned_path, index=False)

cleaned_path


In [13]:
import pandas as pd
import numpy as np

# Load the metrics dataset
df = pd.read_csv('project_metrics_for_tableau.csv')

# Ensure numeric columns are floats
df['success_rate_pct'] = df['success_rate_pct'].astype(float)
df['avg_time_hr'] = df['avg_time_hr'].astype(float)
df['manual_dev_hours'] = df['manual_dev_hours'].astype(float)

# Iterate over weight combinations (success weight from 0.0 to 1.0)
results = []
for w in np.linspace(0, 1, 21):  # 0%, 5%, ..., 100%
    prod_score = (
        w * df['success_rate_pct'] +
        (1 - w) * (1 - df['avg_time_hr'] / df['manual_dev_hours']) * 100
    )
    avg_score = prod_score.mean()
    results.append((w, avg_score))

# Filter combos yielding average between 77% and 94%
valid = [(w, round(avg, 2)) for w, avg in results if 77 <= avg <= 94]

# Display results
print("Weight (success_rate_pct) -> Average Productivity Score")
for w, avg in valid:
    print(f"{w:.2f}                 -> {avg}%")


Weight (success_rate_pct) -> Average Productivity Score
0.45                 -> 78.44%
0.50                 -> 79.92%
0.55                 -> 81.4%
0.60                 -> 82.89%
0.65                 -> 84.37%
0.70                 -> 85.85%
0.75                 -> 87.34%
0.80                 -> 88.82%
0.85                 -> 90.31%
0.90                 -> 91.79%
0.95                 -> 93.27%


In [28]:
import pandas as pd
import numpy as np

# 1. Load cleaned project ideas
df = pd.read_csv('project_ideas_cleaned.csv')

# 2. Generate random project dates within a 6-month window
date_range = pd.date_range(start='2024-10-01', end='2025-04-30')
df['date'] = np.random.choice(date_range, size=len(df))

# 3. manual_dev_hours already present

# 4. Simulate avg_time_hr as 30%–60% of manual_dev_hours
df['avg_time_hr'] = df['manual_dev_hours'] * np.random.uniform(0.2, 0.5, size=len(df))

# 5. success_flag: fewer interruptions, based on avg_time ratio
ratio = df['avg_time_hr'] / df['manual_dev_hours']
df['success_flag'] = np.random.poisson(lam=ratio * 3, size=len(df)).clip(0, 3).astype(int)

# 6. success_rate_pct drops 5% per interruption
df['success_rate_pct'] = np.clip(100 - df['success_flag'] * 5, 0, 100)

# 7. Build monthly cohort users
monthly_new = {
    '2024-10': 200, '2024-11': 500, '2024-12': 1000,
    '2025-01': 2000, '2025-02': 3000, '2025-03': 4000,
    '2025-04': 5000
}
cohort_users, user_counter = {}, 0
for month, count in monthly_new.items():
    month_ts = pd.to_datetime(month)
    users = [f"U{uid:05d}" for uid in range(user_counter+1, user_counter+1+count)]
    cohort_users[month_ts] = users
    user_counter += count

# 8. Simulate a signup + revisit event stream
events = []
end_date = pd.Timestamp('2025-04-30')

for signup_month, users in cohort_users.items():
    # Spread signups across that month
    month_days = pd.date_range(signup_month, signup_month + pd.offsets.MonthEnd(0))
    for u in users:
        signup = np.random.choice(month_days)
        events.append({'user_id': u, 'date': signup})

        # Force ~40% of users to return once within 5–30 days
        if np.random.rand() < 0.4:
            forced = signup + pd.Timedelta(days=int(np.random.uniform(5, 30)))
            if forced <= end_date:
                events.append({'user_id': u, 'date': forced})

        # Further visits (mean inter-arrival = 15 days)
        next_visit = signup
        while True:
            interval = np.random.exponential(scale=15)
            next_visit = next_visit + pd.Timedelta(days=interval)
            if next_visit > end_date:
                break
            events.append({'user_id': u, 'date': next_visit})

# Build and clean the event DataFrame
event_df = (
    pd.DataFrame(events)
      .drop_duplicates(['user_id','date'])
      .assign(date=lambda d: pd.to_datetime(d['date']))
)

# 9. Assign each project in df to a user_id based on matching event dates
def sample_user_for_date(proj_date):
    pool = event_df[event_df['date'] == proj_date]['user_id'].values
    if len(pool) == 0:
        # if no exact-match, fall back to random user
        return np.random.choice(event_df['user_id'])
    return np.random.choice(pool)

df['user_id'] = df['date'].apply(sample_user_for_date)

# 10. Recompute repeat_flag per user: any visit within 30 days of previous
def mark_repeats(g):
    g = g.sort_values('date').copy()
    g['gap_days'] = g['date'].diff().dt.days.fillna(999)
    g['repeat_flag'] = (g['gap_days'] <= 30).astype(int)
    return g.drop(columns='gap_days')

df = df.groupby('user_id', group_keys=False).apply(mark_repeats).reset_index(drop=True)

# 11. Cost calculations
df['baseline_cost_usd'] = df['manual_dev_hours'] * 50    # $50/hr dev rate
df['esh_cost_usd']      = df['avg_time_hr'] * 1          # $1/hr platform cost

# 12. Productivity Score (using your 70/30 weighting)
df['productivity_score'] = (
    0.7 * df['success_rate_pct'] +
    0.3 * (1 - df['avg_time_hr'] / df['manual_dev_hours']) * 100
).clip(0, 100)

# 13. Cost savings
df['cost_savings_usd'] = df['baseline_cost_usd'] - df['esh_cost_usd']

# 14. Save to CSV
output_path = 'project_metrics_for_tableau1.csv'
df.to_csv(output_path, index=False)

print("✅ Generated:", output_path)


✅ Generated: project_metrics_for_tableau1.csv


C:\Users\Gautam\AppData\Local\Temp\ipykernel_13500\3628349513.py:86: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('user_id', group_keys=False).apply(mark_repeats).reset_index(drop=True)


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Load your metrics DataFrame
df = pd.read_csv('project_metrics_for_tableau1.csv')

# 1) Reassign `id` in strict chronological order
df = df.sort_values('date').reset_index(drop=True)
df['id'] = np.arange(1, len(df) + 1)

# 2) “Smooth” user assignment:
#    – Cap consecutive projects per user at 5 in any rolling 7‑day window
#    – If cap reached, force a “cool‑off” of 7–30 days before next assignment

def smooth_user_runs(df):
    df = df.sort_values(['user_id', 'date']).copy()
    next_allowed = {}   # user_id -> earliest next date allowed
    new_rows = []

    for _, row in df.iterrows():
        uid = row['user_id']
        dt  = pd.to_datetime(row['date'])

        # initialize
        if uid not in next_allowed:
            next_allowed[uid] = pd.Timestamp.min

        # if user is “too soon,” push date forward to cool-off window
        if dt < next_allowed[uid]:
            cool_days = np.random.randint(7, 31)
            dt = next_allowed[uid] + timedelta(days=cool_days)

        # update next_allowed based on 7‑day cap
        # count how many events in last 7 days for this uid
        recent = [r for r in new_rows if r['user_id']==uid and (dt - r['date']).days < 7]
        if len(recent) >= 5:
            # reached cap → impose immediate cool‑off
            cool_days = np.random.randint(7, 31)
            dt = recent[-1]['date'] + timedelta(days=cool_days)

        # record this adjusted row
        new_rows.append({**row, 'date': dt})

        # set next_allowed threshold (no restriction until after this)
        next_allowed[uid] = dt

    return pd.DataFrame(new_rows)

df = smooth_user_runs(df)

# 3) Recompute `repeat_flag` so only returns ≤30 days count as repeats,
#    and older returns are treated as new adoption (flag=0):
def recompute_repeat_flag(df):
    df = df.sort_values(['user_id', 'date']).copy()
    df['repeat_flag'] = 0
    last_date = {}

    for idx, row in df.iterrows():
        uid = row['user_id']
        dt  = row['date']

        if uid in last_date:
            gap = (dt - last_date[uid]).days
            df.at[idx, 'repeat_flag'] = 1 if gap <= 30 else 0

        last_date[uid] = dt

    return df

df = recompute_repeat_flag(df)

# Save the cleaned & smoothed DataFrame
df = df.sort_values('date').reset_index(drop=True)
df['id'] = df.index + 1

retention = df.groupby('user_id')['repeat_flag'].max().mean() * 100
print(f"New 30-day retention: {retention:.2f}%")

df.to_csv('project_metrics_smoothed.csv', index=False)
print("Saved smoothed data to project_metrics_smoothed.csv")


New 30-day retention: 38.49%
Saved smoothed data to project_metrics_smoothed.csv


In [38]:
print(df['productivity_score'].mean())
print(df['cost_savings_usd'].mean())

85.97507817234957
290.55637150244337


In [39]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Load the smoothed dataset
df = pd.read_csv('project_metrics_smoothed.csv')
df['date'] = pd.to_datetime(df['date'])

# Split into before/after May 1, 2025
cutoff = pd.Timestamp('2025-05-01')
pre  = df[df['date'] < cutoff].copy()
post = df[df['date'] >= cutoff].copy()

# 1) Reassign post‑cutoff dates randomly between May 1–May 20, 2025
post_dates = pd.date_range('2025-05-01', '2025-05-20')
post['date'] = np.random.choice(post_dates, size=len(post), replace=True)

# 2) Recompute repeat_flag on post only (30‑day window against BOTH pre and post)
#    First, combine date streams so we can look back into pre for each user
combined = pd.concat([pre, post], ignore_index=True)
combined = combined.sort_values(['user_id','date']).reset_index(drop=True)

# 3) Recalculate repeat_flag for every row, but only overwrite post rows
last_seen = {}
flags = []
for idx, row in combined.iterrows():
    uid, dt = row['user_id'], row['date']
    if uid in last_seen and (dt - last_seen[uid]).days <= 30:
        flags.append(1)
    else:
        flags.append(0)
    last_seen[uid] = dt
combined['repeat_flag'] = flags

# 4) Split back out and save (pre flags remain unchanged; post flags updated)
pre_final  = combined[combined['date'] < cutoff]
post_final = combined[combined['date'] >= cutoff]

df_final = pd.concat([pre_final, post_final], ignore_index=True)
df_final = df_final.sort_values('date').reset_index(drop=True)
df_final['id'] = np.arange(1, len(df_final)+1)  # reassign sequential IDs

# Write out
df_final.to_csv('project_metrics_after_may_adjusted.csv', index=False)
print("Saved adjusted CSV with updated dates & repeat_flag after May 1, 2025.")


Saved adjusted CSV with updated dates & repeat_flag after May 1, 2025.


In [42]:
retention = df_final.groupby('user_id')['repeat_flag'].max().mean() * 100
print(f"New 30-day retention: {retention:.2f}%")
print(df_final['productivity_score'].mean())
print(df_final['cost_savings_usd'].mean())

New 30-day retention: 38.49%
85.97507817234957
290.55637150244337
